In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.ConvKNRM.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3901.20it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:07<00:00, 2502.55it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 1668858.01it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 5175.27it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:06<00:00, 2861.55it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 506041.01it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 6173.15it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f5722d98470>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f5722d43cf8>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [5]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [7]:
padding_callback = mz.models.ConvKNRM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [8]:
model = mz.models.ConvKNRM()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['filters'] = 128 
model.params['conv_activation_func'] = 'tanh' 
model.params['max_ngram'] = 3
model.params['use_crossmatch'] = True 
model.params['kernel_num'] = 11
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ConvKNRM(
  (embedding): Embedding(30058, 300)
  (q_convs): ModuleList(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 0), value=0)
      (1): Conv1d(300, 128, kernel_size=(1,), stride=(1,))
      (2): Tanh()
    )
    (1): Sequential(
      (0): ConstantPad1d(padding=(0, 1), value=0)
      (1): Conv1d(300, 128, kernel_size=(2,), stride=(1,))
      (2): Tanh()
    )
    (2): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))
      (2): Tanh()
    )
  )
  (d_convs): ModuleList(
    (0): Sequential(
      (0): ConstantPad1d(padding=(0, 0), value=0)
      (1): Conv1d(300, 128, kernel_size=(1,), stride=(1,))
      (2): Tanh()
    )
    (1): Sequential(
      (0): ConstantPad1d(padding=(0, 1), value=0)
      (1): Conv1d(300, 128, kernel_size=(2,), stride=(1,))
      (2): Tanh()
    )
    (2): Sequential(
      (0): ConstantPad1d(padding=(0, 2), value=0)
      (1): Conv1d(300, 128, kernel_size=(3,), stride=(1,))

In [9]:
optimizer = torch.optim.Adadelta(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10,
    scheduler=scheduler,
    clip_norm=10
)

In [10]:
trainer.run()

[Iter-255 Loss-0.729]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4927 - normalized_discounted_cumulative_gain@5(0.0): 0.5658 - mean_average_precision(0.0): 0.5122



[Iter-510 Loss-0.490]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5325 - normalized_discounted_cumulative_gain@5(0.0): 0.5798 - mean_average_precision(0.0): 0.5406



[Iter-765 Loss-0.232]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5023 - normalized_discounted_cumulative_gain@5(0.0): 0.5629 - mean_average_precision(0.0): 0.5198



[Iter-1020 Loss-0.077]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5288 - normalized_discounted_cumulative_gain@5(0.0): 0.5965 - mean_average_precision(0.0): 0.5558



[Iter-1275 Loss-0.037]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5359 - normalized_discounted_cumulative_gain@5(0.0): 0.599 - mean_average_precision(0.0): 0.557



[Iter-1530 Loss-0.024]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5442 - normalized_discounted_cumulative_gain@5(0.0): 0.6005 - mean_average_precision(0.0): 0.5604



[Iter-1785 Loss-0.017]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5452 - normalized_discounted_cumulative_gain@5(0.0): 0.605 - mean_average_precision(0.0): 0.5643



[Iter-2040 Loss-0.014]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5452 - normalized_discounted_cumulative_gain@5(0.0): 0.6052 - mean_average_precision(0.0): 0.5644



[Iter-2295 Loss-0.017]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5457 - normalized_discounted_cumulative_gain@5(0.0): 0.6039 - mean_average_precision(0.0): 0.5627



[Iter-2550 Loss-0.018]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5457 - normalized_discounted_cumulative_gain@5(0.0): 0.6033 - mean_average_precision(0.0): 0.5625

Cost time: 5077.453552007675s
